<a href="https://colab.research.google.com/github/Dererkzzz/ECE1512-2023F-ProjectRepo-Longqian-Zhang-Yuxuan-Liu/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow.compat.v2 as tf #
from typing import Union
from tensorflow.keras.layers import *
from keras.preprocessing.image import ImageDataGenerator
#from keras.utils.np_utils import to_categorical
import tensorflow.keras.utils as utils
import keras
import numpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


tf.enable_v2_behavior()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import os
target_directory = r'/content/drive/My Drive/ECE_1512_ProjectA/mhist_dataset/images'
os.chdir(target_directory)
image_table=pd.read_csv('/content/drive/My Drive/ECE_1512_ProjectA/mhist_dataset/annotations.csv')




Shape of the 'train_set' DataFrame:
(2175, 4)


train dataset


In [12]:
train_set = image_table[image_table['Partition'] == 'train']
print("Shape of the 'train_set' DataFrame:")
print(train_set.shape)

Shape of the 'train_set' DataFrame:
(2175, 4)


test dataset

In [13]:
test_set = image_table[image_table['Partition'] == 'test']
print("Shape of the 'test_set' DataFrame:")
print(test_set.shape)

Shape of the 'test_set' DataFrame:
(977, 4)


In [14]:
image_dir = '/content/drive/My Drive//ECE_1512_ProjectA/mhist_dataset/images'

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1/255.0,      # 将像素值缩放到0到1之间
    shear_range=0.1,      # 错切变换范围
    rotation_range=15,    # 旋转范围，最多旋转15度
    horizontal_flip=True, # 水平翻转
    vertical_flip=True    # 垂直翻转
)

test_datagen = ImageDataGenerator(
    rescale=1/255.0  # 将像素值缩放到0到1之间
)


create dataset generator

In [18]:
# 创建训练数据生成器
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_set,             # 从数据框中读取训练数据
    directory=image_dir,             # 图像所在的目录路径
    x_col="Image Name",              # 图像文件名列
    y_col="Majority Vote Label",     # 类别标签列
    batch_size=32,                   # 批次大小
    seed=42,                         # 随机种子，可使生成的数据可重复
    shuffle=True,                    # 是否随机洗牌数据
    interpolation='bilinear',        # 图像大小调整插值方法
    class_mode="categorical",        # 多分类问题
    target_size=(224, 224)           # 目标图像大小
)


Found 2175 validated image filenames belonging to 2 classes.


In [20]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_set,             # 从数据框中读取训练数据
    directory=image_dir,             # 图像所在的目录路径
    x_col="Image Name",              # 图像文件名列
    y_col="Majority Vote Label",     # 类别标签列
    batch_size=32,                   # 批次大小
    seed=42,                         # 随机种子，可使生成的数据可重复
    shuffle=False,                    # 是否随机洗牌数据
    interpolation='bilinear',        # 图像大小调整插值方法
    class_mode="categorical",        # 多分类问题
    target_size=(224, 224)           # 目标图像大小
)


Found 977 validated image filenames belonging to 2 classes.


Create ResNet50v2 model

In [21]:
resnet = tf.keras.applications.ResNet50V2(
    include_top=False,  # 不包括顶层（全连接层）
    weights="imagenet",  # 使用预训练权重
    input_tensor=None,  # 没有指定输入张量，将在模型内部创建
    input_shape=(224, 224, 3),  # 输入图像的形状为 224x224 像素，RGB通道
    pooling='avg'  # 使用全局平均池化层
)

94668760/94668760 [==============================] - 1s 0us/step


In [23]:
# 获取 ResNet 模型
resnet = tf.keras.applications.ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling='avg'
)

# 定义要冻结的层的数量
freeze_layers = 191

# 冻结模型的前 freeze_layers 层
for layer in resnet.layers[:freeze_layers]:
    layer.trainable = False

# 输出已冻结层的数量
print(f"frozen {freeze_layers} layer")

frozen 191 layer


In [ ]:
# 检查模型层的可训练性质和名称
for idx, layer in enumerate(resnet.layers, 1):
    print(f"layer {idx} {layer.trainable},  {layer.name[:5]}")

In [27]:
# 创建全连接输出层
output_layer = tf.keras.layers.Dense(units=2, activation='linear')

# 创建教师模型
teacher_res = tf.keras.Sequential([resnet, output_layer])
#teacher_res.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 dense_1 (Dense)             (None, 2)                 4098      
                                                                 
Total params: 23568898 (89.91 MB)
Trainable params: 4098 (16.01 KB)
Non-trainable params: 23564800 (89.89 MB)
_________________________________________________________________


Create mobilenet model

In [32]:
mobilenet = tf.keras.applications.MobileNetV2(
    include_top=False,  # 不包括顶层（全连接层）
    weights="imagenet",  # 使用预训练权重
    input_tensor=None,  # 没有指定输入张量，将在模型内部创建
    input_shape=(224, 224, 3),  # 输入图像的形状为 224x224 像素，RGB通道
    pooling='max'
)

9406464/9406464 [==============================] - 0s 0us/step


In [38]:
freeze_layers_start = 'block_16'  # 指定要从哪一层开始冻结
for layer in mobilenet.layers:
    if layer.name.startswith(freeze_layers_start):
        break
    layer.trainable = False

# 输出已冻结层的数量
num_trainable_layers = len([layer for layer in mobilenet.layers if layer.trainable])
print(f"frozen {len(mobilenet.layers) - num_trainable_layers} layer")

frozen 143 layer


In [ ]:
for idx, layer in enumerate(mobilenet.layers, 1):
    trainable_status = "Trainable" if layer.trainable else "Frozen"
    layer_name = layer.name[:8]  # 获取前8个字符的名称
    print(f"layer{idx}: {trainable_status} {layer_name}")

In [41]:
output_layer = tf.keras.layers.Dense(units=2, activation='linear')

# 创建学生模型
student_mobile = tf.keras.Sequential([mobilenet, output_layer])